In [1]:
import numpy as np
import cmath
import math
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [2]:
res = 2160
view = np.array([[-2, 1], [-1.5, 1.5]])

def function(z, c):
	return z**2 + c

def recursive(f, n):
	def f1(x, *args):
		for i in range(n):
			x = f(x, *args)
		return x
	return f1



cc = 0
pts = 1_000_000
its = 1_000

In [ ]:
l=[]
cc = 0
for i in tqdm(range(pts)):
    z = np.random.rand(2) * (view[:,1] - view[:,0]) + view[:, 0]
    z = z[0]+ 1j*z[1]
    c = z

    if (np.abs(c)**2) * (8 * np.abs(c)**2 - 3) <= 3/32 - c.real:
        continue
    
    z = 0

    n = 0
    l1 = []
    while np.abs(z) < 2 and n < its:
        z = function(z, c)
        n += 1
        l1.append(np.array((z.real, z.imag, n)))
    if np.abs(z) >= 2:
        cc += 1
        l.extend(l1)
print(cc/(pts))

In [ ]:
l=[]
cc = 0
z = np.random.rand(pts, 2) * (view[:,1] - view[:,0]) + view[:, 0]
z = z[:,0]+ 1j*z[:,1]
c = np.copy(z)

x = np.logical_not((np.abs(c)**2) * (8 * np.abs(c)**2 - 3) <= 3/32 - c.real)
c = c[x]

z = np.zeros_like(c)
n = 0
l1 = []

ind = np.arange(z.shape[0])

for i in tqdm(range(its)):
    z = function(z, c)
    n += 1
    l1.append([z.copy(), ind.copy(), n])

    x = np.abs(z) < 2
    c = c[x]
    z = z[x]
    ind = ind[x]

In [ ]:
print(len(l1))
l = []
ll = []
for a in tqdm(l1):
    x = np.logical_not(np.isin(a[1], ind))
    l2 = np.stack((a[0][x].real, a[0][x].imag, np.full(a[0][x].shape, a[2]))).T
    l.extend(l2.tolist())

ll = [[e[0] for e in l], [e[1] for e in l], [e[2] for e in l]]
print(len(l))

In [ ]:
print(l[550000])

In [ ]:
density = np.zeros((res, res), dtype=np.float32)
density2 = np.zeros((res, res), dtype=np.float32)
for z in tqdm(l):
    zz = z[0:2]
    it = z[2]
    n = res * (zz - view[:, 0]) / (view[:,1] - view[:,0])

    if (n >= res - 1).any() or (n < 0).any():
        continue

    x, y = np.floor(n)
    try:
        x, y = int(x), int(y)
    except ValueError:
        continue

    if True:
        density2[x, y] += it
        density[x, y] += 1
    else:
        xF = float(n[0])%1.0
        yF = float(n[1])%1.0
        density[x, y] += (xF + yF)/4
        density[x + 1, y] += (1 - xF + yF)/4
        density[x, y + 1] += (xF + 1 - yF)/4
        density[x + 1, y + 1] += (2 - xF - yF)/4

In [ ]:
density = np.zeros((res, res), dtype=np.float32)
density2 = np.zeros((res, res), dtype=np.float32)



In [ ]:
#final = np.mod(density2, 20)/density
final = density2.copy()/30000
final = np.sqrt(final)
#final = np.log(1+final)

vmax = 1
plt.imshow(final, cmap='Greys_r', vmin=0, vmax=vmax)
plt.show()

In [ ]:
plt.imsave('t13.png',final, cmap='Greys_r', vmin=0, vmax=vmax)

In [ ]:
import pickle
with open('saved.dat', "wb") as f:
    pickle.dump(l, f)